This notebook is to be run in Kaggle. To access the data, please add the dataset from the following link www.kaggle.com/dataset/7a3aa98e18ee9bc22957ef4f78ac001358c3b3a9c69ea4d99366a8c7fa0e144d to your kernel/Kaggle account and the trained model weights included with this notebook to see predictions without training.

# Importing libraries

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

# Parameters

In [ ]:
x_wid, y_wid = (224, 224) # (96,96)
n_channels = 3
batch_size = 64

In [ ]:
%ls '../input'

In [ ]:
train_dir = '../input/fhisto/train'
val_dir = '../input/fhisto/val'
test_dir = '../input/fhisto/test'

# Data Generator

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import preprocess_input

In [ ]:
train_datagen = ImageDataGenerator(
        preprocessing_function = preprocess_input,
        horizontal_flip=True,
        vertical_flip=True
        )

val_datagen = ImageDataGenerator(
        preprocessing_function = preprocess_input)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(x_wid, y_wid),
        color_mode='rgb',
        batch_size=batch_size,
        class_mode='binary'
        )

validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(x_wid, y_wid),
        batch_size=batch_size,
        color_mode='rgb',
        class_mode='binary'
        )

In [ ]:
import matplotlib.pyplot as plt
import random
#plotting rondom images from dataset
def class_plot(data , n_figures = 12):
    n_row = int(n_figures/4)
    fig,axes = plt.subplots(figsize=(14, 10), nrows = n_row, ncols=4)
    for ax in axes.flatten():
        a = random.randint(0,len(data))
        (img,lbl) = data[a]
        #print(lbl)
        for label,image in zip(lbl,img):
          if label==1.:
            l='malignant'
          elif label==0.:
            l='benign'
          image=np.array(image).astype(np.uint8)     
          im = ax.imshow(image)
          ax.set_title(l)
          ax.axis('off')
        pass
        
    plt.show()
class_plot(train_generator)

# Load Model

In [ ]:
from keras.layers import Input, GlobalAveragePooling2D, GlobalMaxPooling2D, Concatenate, BatchNormalization, Dropout, Dense
from keras.models import Model
from keras.applications.resnet import ResNet50
from keras.optimizers import Adam

In [ ]:
sz = (x_wid,y_wid,n_channels)
base_model = ResNet50(weights="imagenet", input_shape=sz, input_tensor=Input(sz), include_top=False)

model_name='resnet50'

avg = GlobalAveragePooling2D()(base_model.output)
mx = GlobalMaxPooling2D()(base_model.output)
out = Concatenate()([avg, mx])
out = BatchNormalization()(out)
out = Dropout(0.25)(out)
out = Dense(512, activation="relu")(out)
out = BatchNormalization()(out)
out = Dropout(0.5)(out)
out = Dense(1, activation="sigmoid")(out)
model = Model(inputs=base_model.input, outputs=out)

for layer in base_model.layers:    
    if layer.name[-2:]=='bn':
        layer.trainable = True
    else:
        layer.trainable = False

optimizer = Adam(lr=0.01)
model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
model.summary()

# Callbacks

In [ ]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, CSVLogger
filepath = model_name+'_full_weights.{epoch:02d}-{val_loss:.2f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=False, save_weights_only=False, mode='auto', period=1)
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='auto', min_delta=0.0001)
early = EarlyStopping(monitor="val_loss", mode="min", patience=20)
csv_logger = CSVLogger('log'+model_name+'.csv', append=True, separator=';')
    
callbacks_list = [csv_logger, checkpoint, reduceLROnPlat, early]

# Train Model

In [ ]:
loss_history = model.fit_generator(train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = 30,
    callbacks = callbacks_list)

In [ ]:
def plot_history(history):
    loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' not in s]
    val_loss_list = [s for s in history.history.keys() if 'loss' in s and 'val' in s]
    acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' not in s]
    val_acc_list = [s for s in history.history.keys() if 'acc' in s and 'val' in s]
    
    if len(loss_list) == 0:
        print('Loss is missing in history')
        return 
    
    ## As loss always exists
    epochs = range(1,len(history.history[loss_list[0]]) + 1)
    
    ## Loss
    plt.figure(1)
    for l in loss_list:
        plt.plot(epochs, history.history[l], 'b', label='Training loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    for l in val_loss_list:
        plt.plot(epochs, history.history[l], 'g', label='Validation loss (' + str(str(format(history.history[l][-1],'.5f'))+')'))
    
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    ## Accuracy
    plt.figure(2)
    for l in acc_list:
        plt.plot(epochs, history.history[l], 'b', label='Training accuracy (' + str(format(history.history[l][-1],'.5f'))+')')
    for l in val_acc_list:    
        plt.plot(epochs, history.history[l], 'g', label='Validation accuracy (' + str(format(history.history[l][-1],'.5f'))+')')

    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

# Plot Loss

In [ ]:
plot_history(loss_history)

# Prediction

In [ ]:
model.load_weights('../input/resnet50weights.30-0.13.hdf5.hdf5')

In [ ]:
import pandas as pd
from keras.preprocessing.image import load_img, img_to_array

prediction_filename=model_name+'test_pred'
threshold=0.5
test_samples=os.listdir(test_dir)
ID=[]
test_predictions=[]

for i in test_samples:
    image = load_img(os.path.join(test_dir,i), target_size=(224, 224))
    print(i)
    # convert the image to an array
    img = img_to_array(image)
    print(img.shape)
    # expand dimensions so that it represents a single 'sample'

    image_norm = np.expand_dims(img, axis=0)
    image_norm = preprocess_input(image_norm)
    pred = model.predict(image_norm)
    print(pred[0,0])
    if pred > threshold:
        lbl=1
    else:
        lbl=0
    ID.append(str(i))
    test_predictions.append(lbl)

TP=pd.DataFrame(list(zip(ID, test_predictions)),
columns=['id','prediction'])
TP.to_csv(prediction_filename+".csv", encoding='utf-8', index=False)

# Pytorch

In [ ]:
!pip install torchsummary

import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from torchsummary import summary
from torch.autograd import Variable

import os
import numpy as np
import pandas as pd
import cv2
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm.autonotebook import tqdm
def normalization_parameter(dataloader):
    mean = 0.
    std = 0.
    nb_samples = len(dataloader.dataset)
    for data,_ in tqdm(dataloader):
        batch_samples = data.size(0)
        data = data.view(batch_samples, data.size(1), -1)
        mean += data.mean(2).sum(0)
        std += data.std(2).sum(0)
    mean /= nb_samples
    std /= nb_samples
    return mean.numpy(), std.numpy()

In [ ]:
batch_size = 128
mean = np.array([0.70039797, 0.5381607, 0.6912888])
std = np.array([0.18225521, 0.20144886, 0.16523378])

## Transformations

In [ ]:
train_transform = transforms.Compose([
                                        transforms.RandomHorizontalFlip(), 
                                        transforms.RandomVerticalFlip(),
                                        transforms.RandomRotation(20),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean,std)])
test_transform = transforms.Compose([
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean,std)])
#inverse normalization for image plot
inv_normalize = transforms.Normalize(
    mean=-1*np.divide(mean,std),
    std=1/std
)

## Data Loader

In [ ]:
def data_loader(train_data, test_data, valid_size, batch_size=128):
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    
    data_len = len(test_data)
    indices = list(range(data_len))
    np.random.shuffle(indices)
    
    split1 = int(np.floor(valid_size * data_len))
    valid_idx, test_idx = indices[:split1], indices[split1:]
    
    valid_sampler = SubsetRandomSampler(valid_idx)
    test_sampler = SubsetRandomSampler(test_idx)
    
    valid_loader = DataLoader(test_data, batch_size= batch_size, sampler=valid_sampler)
    test_loader = DataLoader(test_data, batch_size= batch_size, sampler=test_sampler)
    
    loader = {'train': train_loader, 'val': valid_loader, 'test': test_loader}
    
    return loader

In [ ]:
train_data = ImageFolder(root='../input/histopathology/train', transform=train_transform)
test_data = ImageFolder(root='../input/histopathology/val', transform=test_transform)

loader = data_loader(train_data, test_data, valid_size=0.2, batch_size=batch_size)

#label of classes
classes = train_data.classes
#encoder and decoder to convert classes into integer
decoder = {}
for i in range(len(classes)):
    decoder[classes[i]] = i
encoder = {}
for i in range(len(classes)):
    encoder[i] = classes[i]

## Plot Images

In [ ]:
import random
#plotting rondom images from dataset
def class_plot(data, encoder, inv_normalize = None, n_figures = 12):
    n_row = int(n_figures/4)
    fig,axes = plt.subplots(figsize=(14, 10), nrows = n_row, ncols=4)
    for ax in axes.flatten():
        a = random.randint(0,len(data))
        (image,label) = data[a]
        print(type(image))
        label = int(label)
        l = encoder[label]
        if(inv_normalize!=None):
            image = inv_normalize(image)
        
        image = image.numpy().transpose(1,2,0)
        im = ax.imshow(image)
        ax.set_title(l)
        ax.axis('off')
    plt.show()
class_plot(train_data,encoder,inv_normalize)

## Set GPU

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.is_available()

## Early Stopping

In [ ]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, verbose=True):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), 'checkpoint.pt')
        self.val_loss_min = val_loss

## Cyclical LR

In [ ]:
def cyclical_lr(stepsize, min_lr=3e-4, max_lr=3e-3):

    # Scaler: we can adapt this if we do not want the triangular CLR
    scaler = lambda x: 1.

    # Lambda function to calculate the LR
    lr_lambda = lambda it: min_lr + (max_lr - min_lr) * relative(it, stepsize)

    # Additional function to see where on the cycle we are
    def relative(it, stepsize):
        cycle = math.floor(1 + it / (2 * stepsize))
        x = abs(it / stepsize - 2 * cycle + 1)
        return max(0, (1 - x)) * scaler(cycle)

    return lr_lambda

## Plot Metrics

In [ ]:
def error_plot(loss):
    plt.figure(figsize=(10,5))
    plt.plot(loss)
    plt.title("Training loss plot")
    plt.xlabel("epochs")
    plt.ylabel("Loss")
    plt.show()
def acc_plot(acc):
    plt.figure(figsize=(10,5))
    plt.plot(acc)
    plt.title("Training accuracy plot")
    plt.xlabel("epochs")
    plt.ylabel("accuracy")
    plt.show()
# To plot the wrong predictions given by model
def wrong_plot(n_figures,true,ima,pred,encoder,inv_normalize):
    print('Classes in order Actual and Predicted')
    n_row = int(n_figures/3)
    fig,axes = plt.subplots(figsize=(14, 10), nrows = n_row, ncols=3)
    for ax in axes.flatten():
        a = random.randint(0,len(true)-1)
    
        image,correct,wrong = ima[a],true[a],pred[a]
        image = torch.from_numpy(image)
        correct = int(correct)
        c = encoder[correct]
        wrong = int(wrong)
        w = encoder[wrong]
        f = 'A:'+c + ',' +'P:'+w
        if inv_normalize !=None:
            image = inv_normalize(image)
        image = image.numpy().transpose(1,2,0)
        im = ax.imshow(image)
        ax.set_title(f)
        ax.axis('off')
    plt.show()
def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = metrics.confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax
def performance_matrix(true,pred):
    precision = metrics.precision_score(true,pred,average='macro')
    recall = metrics.recall_score(true,pred,average='macro')
    accuracy = metrics.accuracy_score(true,pred)
    f1_score = metrics.f1_score(true,pred,average='macro')
    print('Precision: {} Recall: {}, Accuracy: {}: ,f1_score: {}'.format(precision*100,recall*100,accuracy*100,f1_score*100))

## Training Parameters

In [ ]:
num_epochs = 20
lr = 0.0005

## Load Model

In [ ]:
resnet = models.resnet34().to(device)
print(resnet)

## Fine-tune Layers

In [ ]:
from torch import nn

# https://docs.fast.ai/layers.html#AdaptiveConcatPool2d
class AdaptiveConcatPool2d(nn.Module):
    def __init__(self, size=1):
        super().__init__()
        self.output_size = size
        self.ap = nn.AdaptiveAvgPool2d(size)
        self.mp = nn.AdaptiveMaxPool2d(size)
        
    def forward(self, x):
        return torch.cat([self.mp(x), self.ap(x)], 1)

# https://docs.fast.ai/layers.html#Flatten
class Flatten(nn.Module):
    def forward(self, x): return x.view(x.size(0), -1)
    
in_features = resnet.fc.in_features * 2    
num_hidden = 512

head = nn.Sequential(
    AdaptiveConcatPool2d(1),
    Flatten(),
    nn.BatchNorm1d(in_features),
    nn.Dropout(0.5),
    nn.Linear(in_features=in_features, out_features=num_hidden),
    nn.ReLU(),
    nn.BatchNorm1d(num_hidden),
    nn.Dropout(0.5),
    nn.Linear(in_features=num_hidden, out_features=2),
)

model = nn.Sequential(
    nn.Sequential(*list(resnet.children())[:-2]),
    head
)

model.to(device)

## Training & Validation Step

In [ ]:
def train(train_loader, model, criterion, optimizer, scheduler, cyclic):
    total_loss = 0.0
    num_batches = len(train_loader)
    size = num_batches * batch_size
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        print(f"Training: {i}/{num_batches}", end="\r")
        
        scheduler.step()
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images) # forward pass
        loss = criterion(outputs, labels)
        total_loss += loss.item() * images.size(0)
        loss.backward()  # backprogagation
        optimizer.step()
        if cyclic:
            # Update LR
            scheduler.step()
        
    return total_loss / size

def validate(valid_loader, model, criterion):
    model.eval()
    with torch.no_grad():
        total_correct = 0
        total_loss = 0.0
        num_batches = len(valid_loader)
        size = num_batches * batch_size
        for i, (images, labels) in enumerate(valid_loader):
            print(f"Validation: {i}/{num_batches}", end="\r")
            
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            total_correct += torch.sum(preds == labels.data)
            total_loss += loss.item() * images.size(0)
            
        return total_loss / size, total_correct.double() / size

## Train Model

In [ ]:
def fit(model, num_epochs, train_loader, valid_loader):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-2, steps_per_epoch=len(train_loader), epochs=num_epochs)
    cyclic = False
    if cyclic:
        factor=10
        end_lr=0.001
        step_size = 4*len(loader['train'])
        clr = cyclical_lr(step_size, min_lr=end_lr/factor, max_lr=end_lr)
        scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, [clr])

        # Make lists to capture the logs
        lr_find_lr = []
    print("epoch\ttrain loss\tvalid loss\taccuracy")
    for epoch in range(num_epochs):
        train_loss = train(train_loader, model, criterion, optimizer, scheduler, cyclic)
        valid_loss, valid_acc = validate(valid_loader, model, criterion)
        print(f"{epoch}\t{train_loss:.5f}\t\t{valid_loss:.5f}\t\t{valid_acc:.3f}")

In [ ]:
model = model.to(device)
fit(model, 20, loader['train'], loader['val'])